# Data processing

## Load data

In [81]:
from utils import *

import matplotlib.pyplot as plt
import seaborn as sb
from copy import deepcopy

%matplotlib inline

account_df = read_to_df("account.csv")
card_test_df = read_to_df("card_test.csv")
card_train_df = read_to_df("card_train.csv")
client_df = read_to_df("client.csv")
disp_df = read_to_df("disp.csv")
district_df = read_to_df("district.csv")
loan_test_df = read_to_df("loan_test.csv")
loan_train_df = read_to_df("loan_train.csv")
trans_test_df = read_to_df("trans_test.csv")
trans_train_df = read_to_df("trans_train.csv")

<ipython-input-81-804235880b95>:18: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  trans_train_df = read_to_df("trans_train.csv")


## Process data

### Process account data

In [82]:
avg_transactions_per_week = 3
avg_weeks_per_month = (365.25 / 7 / 12)

account_df['frequency'] = account_df['frequency'].apply(lambda x: 1 if x == 'monthly issuance' else avg_weeks_per_month if x == 'weekly issuance' else (365.25 / 7 / 12) * avg_transactions_per_week)
account_df["date"] = account_df["date"].apply(lambda x: read_date(x))
account_df.rename(columns={"date": "acc_creation_date", "frequency": "issuance_frequency_per_month"}, inplace=True)

account_df.head()

,account_id,district_id,issuance_frequency_per_month,acc_creation_date
0,576,55,1.0,1993-01-01
1,3818,74,1.0,1993-01-01
2,704,55,1.0,1993-01-01
3,2378,16,1.0,1993-01-01
4,2632,24,1.0,1993-01-02


### Process client data

In [83]:
client_df["sex"] = client_df["birth_number"].apply(lambda x: 0 if int(str(x)[2:4]) > 50 else 1)
client_df["age"] = client_df["birth_number"].apply(lambda x: calculate_age(read_date(x)))

client_df.drop("birth_number", inplace=True, axis=1)

client_df.head()

,client_id,district_id,sex,age
0,1,18,0,28
1,2,1,1,53
2,3,1,0,58
3,4,5,1,42
4,5,5,0,38


### Process disposition data

In [84]:
disp_df.rename(columns={"type": "is_owner"}, inplace=True)
disp_df["is_owner"].replace({"OWNER": True, "DISPONENT": False}, inplace=True)

# Count number clients per account
client_count_df = disp_df.groupby("account_id", as_index=False, group_keys=False).agg(client_count_per_acc=("is_owner", "count"))

disp_df = disp_df.merge(client_count_df, on="account_id")
disp_df = disp_df[disp_df["is_owner"] == True] 
disp_df.drop("is_owner", axis=1, inplace=True)

disp_df.head()

,disp_id,client_id,account_id,client_count_per_acc
0,1,1,1,1
1,2,2,2,2
3,4,4,3,2
5,6,6,4,1
6,7,7,5,1


### Process transaction data

In [85]:
dataframes = [trans_train_df, trans_test_df]

for i in range(len(dataframes)):
    dataframes[i]["operation"].replace(
        {
            "credit in cash": 1,
            "collection from another bank": 2,
            "withdrawal in cash": 3,
            "remittance to another bank": 4,
            "credit card withdrawal": 5,
            "interest credited": 6
        },
        inplace=True
    )

    # Convert "withdrawal in cash" to "withdrawal" in type column
    dataframes[i].loc[dataframes[i]["type"] == "withdrawal in cash", "type"] = "withdrawal"

    # Withdrawal amounts should be negative
    dataframes[i].loc[dataframes[i]["type"] == "withdrawal", "amount"] *= -1

    dataframes[i].drop(["k_symbol", "bank"], axis=1, inplace=True)

trans_train_df, trans_test_df = dataframes

trans_train_df.head()

,trans_id,account_id,date,type,operation,amount,balance,account
0,1548749,5270,930113,credit,1.0,800.0,800.0,NaN
1,1548750,5270,930114,credit,2.0,44749.0,45549.0,80269753.0
2,3393738,11265,930114,credit,1.0,1000.0,1000.0,NaN
3,3122924,10364,930117,credit,1.0,1100.0,1100.0,NaN
4,1121963,3834,930119,credit,1.0,700.0,700.0,NaN


### Process card data

In [86]:
dataframes = [card_train_df, card_test_df]

for i in range(len(dataframes)):
    dataframes[i]["type"].replace({"classic": 1, "junior": 2, "gold": 3}, inplace=True)

card_train_df, card_test_df = dataframes

card_train_df.head()

,card_id,disp_id,type,issued
0,1005,9285,1,931107
1,104,588,1,940119
2,747,4915,1,940205
3,70,439,1,940208
4,577,3687,1,940215


## Generate feature from transaction data

In [87]:
transactions = (trans_train_df, trans_test_df)
account_features = [1, 2]

for i in range(len(transactions)):
    
    # Sorting transactions by date to figure out the most recent balance
    account_features[i] = transactions[i].sort_values(by="date", axis=0, ascending=False)
    account_features[i].drop_duplicates(subset='account_id', keep='first', inplace=True)

    account_features[i].drop(account_features[i].columns.difference(['account_id','balance']), axis=1, inplace=True)
    account_features[i].rename(columns={'balance' : 'final_amount'}, inplace=True)


account_features[0]

,account_id,final_amount
396684,1770,68162.9
393038,3039,32166.2
393030,3076,24233.8
393031,3045,28237.6
393032,3030,21804.5
...,...,...
1716,11013,41142.9
1155,1843,34307.3
1091,9188,20272.8
965,1801,52208.9


## Merge data

In [88]:
# joined_loan_train_df = loan_train_df.merge(right=disp_df, on="account_id", how="left")
# joined_loan_train_df = joined_loan_train_df.merge(right=card_train_df, on="disp_id", how="left")
# joined_loan_train_df.rename(columns={"type_x": "card_type"}, inplace=True)

# # Replace text with numeric data
# joined_loan_train_df["card_type"].replace(
#     {"classic": 1, "junior": 2, "gold": 3}, inplace=True)
# joined_loan_train_df["type_y"].replace({"OWNER": 1, "DISPONENT": 2}, inplace=True)

# joined_loan_train_df.head(50)
# joined_loan_train_df.count() #Doesn't count NA values

# #Join with account
# joined_client__disp_account_df = account_df_processed.merge(right=disp_df, on="account_id", how="right")

# #Extract number of clients
# joined_client__disp_account_df.drop(['district_id', 'date', 'disp_id', 'type'], axis=1, inplace=True)
# #joined_client__disp_account_df = joined_client__disp_account_df.groupby(by=["account_id"]).count()

# joined_client__disp_account_df = joined_client__disp_account_df.groupby(["account_id", "frequency"]).agg({'client_id': 'count'}).reset_index()

# joined_client__disp_account_df.rename(columns = {'client_id' : 'num_clients'}, inplace=True, errors="raise")





# joined_loan_train_df = joined_loan_train_df.merge(right=joined_client__disp_account_df, on='account_id')

# joined_loan_train_df = joined_loan_train_df.merge(right=trans_train_df_most_recent, how='left', on='account_id')

# joined_loan_train_df.count()

In [89]:
loan_dfs = [loan_train_df, loan_test_df]
trans_dfs = (trans_train_df, trans_test_df)
cards_dfs = (card_train_df, card_test_df)

for i in range(len(loan_dfs)):
    # Merge with dispositions
    # loan_dfs[i] = disp_df.merge(loan_dfs[i], on="account_id")
    loan_dfs[i] = loan_dfs[i].merge(disp_df, on="account_id", how="left")

    # # Merge with accounts
    loan_dfs[i] = loan_dfs[i].merge(account_df, on="account_id")

    # Merge with clients
    loan_dfs[i] = loan_dfs[i].merge(client_df, on="client_id", suffixes=["_account", "_client"])

    # Merge with districts
    loan_dfs[i] = loan_dfs[i].merge(district_df, left_on="district_id_client", right_on="code")

    # Merge with districts
    loan_dfs[i] = loan_dfs[i].merge(cards_dfs[i], on="disp_id", how="left")

    # # Merge with transactions
    loan_dfs[i] = loan_dfs[i].merge(account_features[i], on="account_id")




loan_train_df, loan_test_df = loan_dfs

# loan_train_df
with pd.option_context('display.max_rows', 15, 'display.max_columns', None): 
    display(loan_train_df)

,loan_id,account_id,date,amount,duration,payments,status,disp_id,client_id,client_count_per_acc,district_id_account,issuance_frequency_per_month,acc_creation_date,district_id_client,sex,age,code,name,region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,card_id,type,issued,final_amount
0,5314,1787,930705,96396,12,8033,-1,2166,2166,1,30,4.348214,1993-03-22,30,0,51,30,Sokolov,west Bohemia,94812,15,13,8,2,10,81.8,9650,3.38,3.67,100,2985,2804,NaN,NaN,NaN,20100.0
1,5316,1801,930711,165960,36,4610,1,2181,2181,1,46,1.000000,1993-02-13,46,1,30,46,Nachod,east Bohemia,112709,48,20,7,3,10,73.5,8369,1.79,2.31,117,2854,2618,NaN,NaN,NaN,52208.9
2,6456,7123,931209,47016,12,3918,1,8598,8598,1,64,13.044643,1993-04-14,46,0,41,46,Nachod,east Bohemia,112709,48,20,7,3,10,73.5,8369,1.79,2.31,117,2854,2618,NaN,NaN,NaN,58773.2
3,7249,11065,940612,69360,48,1445,1,13295,13603,1,11,1.000000,1993-03-19,46,1,57,46,Nachod,east Bohemia,112709,48,20,7,3,10,73.5,8369,1.79,2.31,117,2854,2618,NaN,NaN,NaN,38665.0
4,6863,9188,930728,127080,60,2118,1,11006,11314,1,45,1.000000,1993-02-08,45,1,62,45,Jicin,east Bohemia,77917,85,19,6,1,5,53.5,8390,2.28,2.89,132,2080,2122,NaN,NaN,NaN,20272.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,5148,940,960529,284280,60,4738,1,1133,1133,2,37,1.000000,1994-11-15,37,0,40,37,Litomerice,north Bohemia,114006,71,26,6,2,9,62.3,9065,4.46,5.39,123,4147,4166,NaN,NaN,NaN,94617.3
324,6667,8127,960715,155880,24,6495,1,9732,10040,2,37,1.000000,1994-10-13,37,0,30,37,Litomerice,north Bohemia,114006,71,26,6,2,9,62.3,9065,4.46,5.39,123,4147,4166,NaN,NaN,NaN,44564.6
325,5741,3754,961103,68544,24,2856,1,4529,4529,2,37,4.348214,1996-02-19,37,1,22,37,Litomerice,north Bohemia,114006,71,26,6,2,9,62.3,9065,4.46,5.39,123,4147,4166,NaN,NaN,NaN,23759.8
326,7036,10019,960925,194940,36,5415,1,12017,12325,1,8,1.000000,1995-06-15,8,1,34,8,Mlada Boleslav,central Bohemia,112065,95,19,7,1,8,69.4,11277,1.25,1.44,127,5179,4987,NaN,NaN,NaN,34212.3


## Dropping features

- All cards can be dropped as there are only 11 out of the total 328 loans making it very hard or impossible to fill in missing values.

In [90]:
loan_train_df.drop(["card_id", "type", "issued"], axis=1)

,loan_id,account_id,date,amount,duration,payments,status,disp_id,client_id,client_count_per_acc,...,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,final_amount
0,5314,1787,930705,96396,12,8033,-1,2166,2166,1,...,2,10,81.8,9650,3.38,3.67,100,2985,2804,20100.0
1,5316,1801,930711,165960,36,4610,1,2181,2181,1,...,3,10,73.5,8369,1.79,2.31,117,2854,2618,52208.9
2,6456,7123,931209,47016,12,3918,1,8598,8598,1,...,3,10,73.5,8369,1.79,2.31,117,2854,2618,58773.2
3,7249,11065,940612,69360,48,1445,1,13295,13603,1,...,3,10,73.5,8369,1.79,2.31,117,2854,2618,38665.0
4,6863,9188,930728,127080,60,2118,1,11006,11314,1,...,1,5,53.5,8390,2.28,2.89,132,2080,2122,20272.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,5148,940,960529,284280,60,4738,1,1133,1133,2,...,2,9,62.3,9065,4.46,5.39,123,4147,4166,94617.3
324,6667,8127,960715,155880,24,6495,1,9732,10040,2,...,2,9,62.3,9065,4.46,5.39,123,4147,4166,44564.6
325,5741,3754,961103,68544,24,2856,1,4529,4529,2,...,2,9,62.3,9065,4.46,5.39,123,4147,4166,23759.8
326,7036,10019,960925,194940,36,5415,1,12017,12325,1,...,1,8,69.4,11277,1.25,1.44,127,5179,4987,34212.3


## Aggregate data 

In [91]:
keep_cols = ["loan_id", "account_id", "status", "loan_date", "creation_date", "loan_amount", "duration", "payments", "gender", "birthdate", "ownership_count", "district_id", "num_inhabitants", 'num_municipalities_with_inhabitants<499', 'num_municipalities_with_inhabitants_500-1999', 'num_municipalities_with_inhabitants_2000-9999', 'num_municipalities_with_inhabitants>10000', 'num_cities', 'ratio_urban_inhabitants', 'average_salary', 'unemployment_rate_95', 'unemployment_rate_96', 'num_entrepreneurs_per_1000_inhabitants', 'num_crimes_95', 'num_crimes_96']
# keep_cols = ["loan_id", "account_id", "status", "date", "creation_date", "amount_x", "duration", "payments", "sex", "age", "ownership_count", "district_id", "num_inhabitants", 'num_municipalities_with_inhabitants<499', 'num_municipalities_with_inhabitants_500-1999', 'num_municipalities_with_inhabitants_2000-9999', 'num_municipalities_with_inhabitants>10000', 'num_cities', 'ratio_urban_inhabitants', 'average_salary', 'unemployment_rate_95', 'unemployment_rate_96', 'num_entrepreneurs_per_1000_inhabitants', 'num_crimes_95', 'num_crimes_96']
def aggregate(df):
    df = df.groupby(keep_cols, as_index=False, group_keys=False).agg({
        "date" : ["max", "min", age_days],
        "operation":["count", count_a, count_b, count_c, count_d, count_e, count_f, 
                     mean_a, mean_b, mean_c, mean_d, mean_e, mean_f,
                     std_a, std_b, std_c, std_d, std_e, std_f,
                     cov_a, cov_b, cov_c, cov_d, cov_e, cov_f],
        "amount": ["mean","min","max","std","last",np.cov,abs_min,rangev],
        "balance":["mean","min","max","std","last",np.cov,abs_min,rangev],
        "type": [count_withdrawal, count_credit, mean_withdrawal, mean_credit, std_withdrawal, std_credit, cov_withdrawal, cov_credit]
    })
    df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]
    df["days_last_trans_loan"] = (df["loan_date"] - df["date_max"]).dt.days
    df["last_balance_per_loan"] = df["balance_last"] / df["loan_amount"]
    df["max_balance_per_loan"] = df["balance_max"] / df["loan_amount"]
    df["date_age_months"] = df["date_age_days"]/30
    df["balance_per_month"] = df["balance_range"] / df["date_age_months"]
    df["transactions_per_month"]=df["operation_count"] / df["date_age_months"]
    # calculate client age at loan request
    df['owner_age_at_loan'] = (df['loan_date'] - df['birthdate']).astype('<m8[Y]') # cast to years
    df.owner_age_at_loan = df.owner_age_at_loan.astype(int) # convert to int
    df['account_months_at_loan'] = ((df['loan_date'] - df['creation_date']).dt.days)/30 # cast to months
#     df.account_age_at_loan = df.account_age_at_loan.astype(int) # convert to int
    df = df.drop(['birthdate',"creation_date"], axis=1) #remove birthdate
    
    return df

In [92]:
# loan_train_df = aggregate(loan_train_df)
# loan_test_df = aggregate(loan_test_df)


## Export preprocessed dataframes

In [93]:
loan_train_df.to_pickle("../out/train.pkl")
loan_test_df.to_pickle("../out/test.pkl")